In [1]:
#
# Download https://github.com/smutchler/trilabytePyML to a ZIP file and
# place subfoler into C:\
#
# You should have this exact path when done correctly: C:\TrilabytePyML\trilabytePyML\AutoForecast.py

In [2]:
import sys

if 'TrilabytePyML' not in sys.path:
    sys.path.insert(1, 'C:\\TrilabytePyML')

In [3]:
import pandas as pd

pd.options.mode.chained_assignment = None

frame = pd.read_csv('C:\\TrilabytePyML\\samples\\retail_unit_demand.csv')
display(frame)

,SKU,STORE_NUMBER,CAL_YEAR,CAL_MONTH,CAL_DATE,UNIT_DEMAND,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,PREDICTOR_4
0,1100,1150,2016,3,3/1/2016,16.0,1.856107,4.916073,6.350111,0.170545
1,1100,1150,2016,4,4/1/2016,28.0,9.803075,5.647253,40.128298,0.190157
2,1100,1150,2016,5,5/1/2016,35.0,3.598739,15.927959,29.807851,0.006683
3,1100,1150,2016,6,6/1/2016,129.0,1.155522,48.465667,17.236057,0.007743
4,1100,1150,2016,7,7/1/2016,179.0,9.673028,6.603080,39.144862,0.230874
...,...,...,...,...,...,...,...,...,...,...
113,1733,993243,2019,8,8/1/2019,NaN,5.563396,13.576681,12.819330,0.038417
114,1733,993243,2019,9,9/1/2019,NaN,6.663194,43.053491,11.374827,0.144381
115,1733,993243,2020,10,10/1/2020,NaN,2.270636,29.233220,59.000652,0.288833
116,1733,993243,2020,11,11/1/2020,NaN,4.903307,49.802078,27.008816,0.017463


In [4]:
options = dict()
options['method'] = 'Auto' #'Auto','MLR','ARIMA','Prophet'
options['sortColumns'] = ['STORE_NUMBER','SKU','CAL_YEAR','CAL_MONTH']
options['splitColumns'] = ['STORE_NUMBER','SKU']
options['predictorColumns'] = ['X_INDEX','PREDICTOR_1','PREDICTOR_2','PREDICTOR_3','PREDICTOR_4']
options['targetColumn'] = 'UNIT_DEMAND'
options['timestampColumn'] = 'CAL_DATE' #only needed by Prophet model  
options['periodicity'] = 12
options['autoDetectOutliers'] = True
options['outlierStdevMultiplier'] = 3.0
options['seasonalityBandwidth'] = 0.6
options['ridgeAlpha'] = 1.0
options['forceNonNegative'] = True
options['seasonality'] = 'Auto' #'Auto','Additive','Multiplicative'

In [5]:
import trilabytePyML.AutoForecast as af

frame = af.splitFramesAndForecast(frame, options)

display(frame)

Importing plotly failed. Interactive plots will not work.


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,SKU,STORE_NUMBER,CAL_YEAR,CAL_MONTH,CAL_DATE,UNIT_DEMAND,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,PREDICTOR_4,...,X_OUTLIER,X_UNIT_DEMAND,X_LPI,X_FORECAST,X_UPI,X_MAPE,X_RESIDUAL,X_APE,X_ERROR,X_METHOD
0,1100,1150,2016,3,3/1/2016,16.0,1.856107,4.916073,6.350111,0.170545,...,0,16.0,0.000000,22.146917,67.006252,15.836504,6.146917,38.4182,None,Prophet
1,1100,1150,2016,4,4/1/2016,28.0,9.803075,5.647253,40.128298,0.190157,...,0,28.0,0.000000,26.475958,72.963055,15.836504,-1.524042,5.44301,None,Prophet
2,1100,1150,2016,5,5/1/2016,35.0,3.598739,15.927959,29.807851,0.006683,...,0,35.0,10.155356,55.487760,103.156123,15.836504,20.487760,58.5365,None,Prophet
3,1100,1150,2016,6,6/1/2016,129.0,1.155522,48.465667,17.236057,0.007743,...,0,129.0,78.305711,124.089295,169.078036,15.836504,-4.910705,3.80675,None,Prophet
4,1100,1150,2016,7,7/1/2016,179.0,9.673028,6.603080,39.144862,0.230874,...,0,179.0,155.617280,201.981193,250.742877,15.836504,22.981193,12.8387,None,Prophet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,1733,993243,2019,8,8/1/2019,NaN,5.563396,13.576681,12.819330,0.038417,...,0,NaN,1635.121728,1671.850614,1705.626057,15.093023,NaN,NaN,None,Prophet
56,1733,993243,2019,9,9/1/2019,NaN,6.663194,43.053491,11.374827,0.144381,...,0,NaN,1463.599270,1501.970392,1539.772116,15.093023,NaN,NaN,None,Prophet
57,1733,993243,2020,10,10/1/2020,NaN,2.270636,29.233220,59.000652,0.288833,...,0,NaN,1518.383261,1553.813659,1592.316243,15.093023,NaN,NaN,None,Prophet
58,1733,993243,2020,11,11/1/2020,NaN,4.903307,49.802078,27.008816,0.017463,...,0,NaN,959.597615,995.201535,1032.412430,15.093023,NaN,NaN,None,Prophet


In [6]:
frame.to_csv('c:\\temp\\retail_unit_demand_forecast.csv')